In [1]:
import os
import numpy as np
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Arial"

from scipy.signal import savgol_filter
import scipy.interpolate as interp

import glob

In [2]:
def extract_values_from_folder_name(folder_name):
    # Split the folder_name using the correct separator for the operating system
    separator = os.sep
    parts = folder_name.split(separator)

    # Extract the values from the folder name
    try:
        N = float(parts[-2].split('_')[0])
        K = float(parts[-2].split('_')[1])
        A = float(parts[-2].split('_')[2])
        A1 = float(parts[-2].split('_')[3])
        return N, K, A, A1
    except ValueError:
        # Handle cases where folder_name is not in the expected format
        return None

In [4]:
folder_pattern = './2023-07-27/*/*_*_*_*/'
folder_list = glob.glob(folder_pattern)

parameters_list = []
SQ_list = []

Q_interp = (np.arange(100)+1)*0.2

for folder in tqdm(folder_list):
    parameters_list.append(extract_values_from_folder_name(folder))
    data_current = np.loadtxt(folder+'SOQ/soq.txt')
    Q = data_current[1:,0]
    SQ = data_current[1:,1]*16384
    SQ_sm = savgol_filter(SQ, 127,2)
    f_interp = interp.interp1d(Q, SQ_sm, kind='linear', fill_value='extrapolate')
    SQ_interp = f_interp(Q_interp)

    SQ_list.append(SQ_interp)

100%|██████████| 63000/63000 [03:55<00:00, 267.60it/s]


In [5]:
# fig = plt.figure(figsize=(6, 6))#,dpi=192)
# ax = plt.subplot(1, 1, 1)

# ax.plot(Q, SQ)
# ax.plot(Q, SQ_sm)
# ax.plot(Q_interp, SQ_interp)

# # ax.set_xscale('log')

In [6]:
parameters_list = np.array(parameters_list)
SQ_list = np.array(SQ_list)
mdic = {'parameters_list':parameters_list, 'SQ_list':SQ_list}

from scipy.io import savemat

savemat('SQ_twoYukawa_ext.mat',mdic)